In [27]:
%pip install osmium folium geodatasets


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
import folium
import osmium as osm
import pandas as pd
import urllib.request
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon

In [29]:
#you can create shape here https://clydedacruz.github.io/openstreetmap-wkt-playground/
#copy polygon into var wkt_poly

#seattle downtown
# wkt_poly = '-122.3567 47.5933,-122.3139 47.5933,-122.3139 47.6517,-122.3567 47.6517'

#new york manhattan downtown
# wkt_poly = '-74.00239853421225 40.76848093731104,-73.95889190956949 40.752874609831736,-73.97423808462916 40.70989961182363,-73.99736844934523 40.704987174650086,-74.01250212220474 40.69698290095204,-74.0223849727772 40.69959263102922,-74.02202142868191 40.71276213012328,-74.01762143243103 40.73881298686783,-74.01512450538576 40.7547003536466,-74.00239853421225 40.76848093731104'

#sf
# wkt_poly = '-122.53185241587468 37.809559644767006,-122.51502511406065 37.710653461982176,-122.51204497967186 37.70676699804295,-122.36787600686763 37.70685478619541,-122.3456333312914 37.713680481105854,-122.34470397313827 37.74027641177274,-122.37476706258099 37.774478724975836,-122.38975963122566 37.805581044953044,-122.40176055172024 37.81335615786739,-122.43924826368664 37.81356889134105,-122.4675810788144 37.8122783423365,-122.49125005056311 37.805329437469155,-122.49451932787737 37.79418788424093,-122.53185241587468 37.809559644767006'

#sv
wkt_poly = '-122.16301466803996 37.486094621836685,-122.21170388627796 37.412707517346064,-122.16430812142787 37.38434852909499,-122.13205091189592 37.35399235923582,-122.10420128423718 37.34060791527514,-122.0776901207864 37.32213615696999,-122.07062748726456 37.29601352469203,-122.0484747039154 37.27603613974286,-121.9691434688866 37.2752793578006,-121.88139156438409 37.26995962602973,-121.81655913591383 37.28730773167956,-121.79111410863696 37.335420643991895,-121.81434480473398 37.39040580901978,-121.89833717420694 37.421568095242066,-121.9401141721755 37.43385473835333,-121.9943742547184 37.42129866120064,-122.0350014232099 37.42909243556316,-122.07515040412544 37.4453550823531,-122.10126862395553 37.47097268828807,-122.11850931867956 37.48897093733271,-122.13452558498828 37.50144482955908,-122.16301466803996 37.486094621836685'

poly_points = [(float(i.split(' ')[1]), float(i.split(' ')[0])) for i in wkt_poly.split(',')]

current_dir = os.getcwd()
filename = f'{current_dir}/map-data.osm'

In [30]:
map_data = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [[[i[1], i[0]] for i in poly_points]],
        "type": "Polygon"
      }
    }
  ]
}
m = folium.Map()
geo_j = folium.GeoJson(data=map_data, style_function=lambda x: {"fillColor": "orange"})
folium.Popup('location').add_to(geo_j)
geo_j.add_to(m)
m.fit_bounds([[min([i[0] for i in poly_points]), min([i[1] for i in poly_points])], [max([i[0] for i in poly_points]), max([i[1] for i in poly_points])]]) 
m

<h1>Download map data</h1>

In [31]:
print(f"wkt polygon:POLYGON(({','.join([f'{i[1]} {i[0]}' for i in poly_points])}))")
poly = ' '.join([f'{i[0]} {i[1]}' for i in poly_points])
url = f'''http://overpass-api.de/api/interpreter?data=(node(poly:"{poly}");way(poly:"{poly}"););(._;>;);out;'''
url = url.replace(" ", "%20")
%time urllib.request.urlretrieve(url, filename)

wkt polygon:POLYGON((-122.16301466803996 37.486094621836685,-122.21170388627796 37.412707517346064,-122.16430812142787 37.38434852909499,-122.13205091189592 37.35399235923582,-122.10420128423718 37.34060791527514,-122.0776901207864 37.32213615696999,-122.07062748726456 37.29601352469203,-122.0484747039154 37.27603613974286,-121.9691434688866 37.2752793578006,-121.88139156438409 37.26995962602973,-121.81655913591383 37.28730773167956,-121.79111410863696 37.335420643991895,-121.81434480473398 37.39040580901978,-121.89833717420694 37.421568095242066,-121.9401141721755 37.43385473835333,-121.9943742547184 37.42129866120064,-122.0350014232099 37.42909243556316,-122.07515040412544 37.4453550823531,-122.10126862395553 37.47097268828807,-122.11850931867956 37.48897093733271,-122.13452558498828 37.50144482955908,-122.16301466803996 37.486094621836685))
CPU times: user 2.23 s, sys: 3.8 s, total: 6.03 s
Wall time: 59.6 s


('/Users/chaoqinglu/src/office_locations/map-data.osm',
 <http.client.HTTPMessage at 0x1267ad690>)

<h1>Process map data</h1>

In [32]:
valid_building_tags = set(['office', 'commercial', 'university', 'school', 'college', 'bank'])
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []
        self.node_coordinates = {}
        self.missing_nodes = 0

    def tag_office(self, elem, elem_type):
        name = ''
        tag_key_office = ''
        tag_val_office = ''
        is_building = False
        building_tag_val = ''
        
        for tag in elem.tags:
            if tag.k.lower() == 'website':
                continue
            if 'office' in tag.k.lower() and tag_key_office != 'office':
                tag_key_office = tag.k.lower()
                tag_val_office = tag.v.lower()
            if 'office' in tag.v.lower() and tag_val_office != 'office':
                tag_key_office = tag.k.lower()
                tag_val_office = tag.v.lower()
            if ('building'==tag.k):
                is_building = True
                building_tag_val = tag.v.lower()
            if tag.k == 'name':
                name = tag.v
        if tag_key_office != '' or tag_val_office != '' or \
            (is_building and (
                building_tag_val in valid_building_tags or \
                building_tag_val == 'yes' and name != ''
            )):
            points = []
            if elem_type == 'way':
                for n in elem.nodes:
                    if n.ref in self.node_coordinates:
                        points.append(self.node_coordinates[n.ref])
                    else:
                        self.missing_nodes += 1
                        #print(f'missing loc for {n.ref} {elem.id}')
            else:
                points.append((elem.location.lon, elem.location.lat))
            self.osm_data.append([
                elem_type, 
                int(elem.id), 
                name.lower(),
                tag_key_office,
                tag_val_office,
                is_building, 
                building_tag_val,
                ','.join([f'{tag.k.lower()}:{tag.v.lower()}' for tag in elem.tags]),
                points,
            ])

    def node(self, n):
        self.node_coordinates[n.id] = (n.location.lon, n.location.lat)
        self.tag_office(n, "node")
    def way(self, w):
        self.tag_office(w, "way")


osmhandler = OSMHandler()
# scan the input file and fills the handler list accordingly
osmhandler.apply_file(filename)
print(f'missing {osmhandler.missing_nodes} nodes in ways')
# transform the list into a pandas DataFrame
data_colnames = ['type', 'id', 'name', 'tag_key', 'tag_val', 'is_building', 'building_tag', 'tags', 'points']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
df_osm.sort_values(by=['name'])

missing 0 nodes in ways


,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points
8803,way,639145764,,,,True,school,"building:school,ele:61.51,height:3.46","[(-121.9231349, 37.2777541), (-121.9232086, 37..."
4009,way,132296660,,office,company,True,yes,"addr:housenumber:654,addr:street:high street,b...","[(-122.1614154, 37.4427508), (-122.1616556, 37..."
8452,way,563781015,,,,True,school,"building:school,ele:38.33,height:3.40","[(-121.8389414, 37.3303943), (-121.8391476, 37..."
8451,way,563781014,,,,True,school,"building:school,building:prefabricated:yes,ele...","[(-121.8386946, 37.3300083), (-121.8387606, 37..."
8450,way,563781013,,,,True,school,"building:school,building:prefabricated:yes,ele...","[(-121.8387916, 37.330112), (-121.8386623, 37...."
...,...,...,...,...,...,...,...,...,...
8268,way,546549901,"信望愛 faith, hope, and love chinese school",,,True,yes,"addr:city:san jose,addr:housenumber:6350,addr:...","[(-122.0166338, 37.3009343), (-122.0168683, 37..."
1494,node,8813107045,利达快递,amenity,post_office,False,,"addr:city:cupertino,addr:housenumber:10601,add...","[(-122.032704, 37.3141899)]"
9676,way,917702297,基督教義興教育中心;christian righteousness education ce...,,,True,yes,"addr:city:saratoga,addr:housenumber:19888,addr...","[(-122.0311546, 37.2938068), (-122.0311635, 37..."
8301,way,553223583,方舟浸信教會;ark baptist church,,,True,commercial,"addr:city:milpitas,addr:housenumber:380,addr:p...","[(-121.8952005, 37.4066914), (-121.8949566, 37..."


<h1>QAs</h1>

In [33]:
df_osm[['type', 'tag_key', 'tag_val', 'is_building']].value_counts()[:100]

type  tag_key   tag_val                                                   is_building
way                                                                       True           7209
      building  office                                                    True            513
node  office    company                                                   False           252
                yes                                                       False           247
way   office    company                                                   True            180
                                                                                         ... 
node  subject   san josé police officer michael johnson memorial highway  False             2
      office    investment                                                False             2
way   name      fedex office                                              True              2
node  building  office                                              

In [34]:
df_osm['name'].value_counts()[:100]

                                             2971
guadalupe parkway                              62
7-eleven                                       33
the ups store                                  30
h&r block                                      20
                                             ... 
329                                             3
cardenas                                        3
santa clara county social services agency       3
bioabsorbant therapeutics                       3
science building                                3
Name: name, Length: 100, dtype: int64

In [38]:
df_osm[df_osm.apply(lambda x:'linkedin ' in x['name'] or 'linkedin' in x['tags'], axis=1)]

,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points
1583,node,9131017876,sutter hill ventures,office,company,False,,"addr:city:palo alto,addr:housenumber:755,addr:...","[(-122.1427342, 37.4203955)]"
1802,node,10844739563,linkedin building n,office,yes,False,,"addr:city:sunnyvale,addr:housenumber:479,addr:...","[(-122.0348812, 37.3893634)]"
1804,node,10844739565,linkedin building m,office,yes,False,,"addr:city:sunnyvale,addr:housenumber:845,addr:...","[(-122.0350464, 37.3916454)]"
1805,node,10844739566,linkedin,office,yes,False,,"addr:city:sunnyvale,addr:housenumber:815,addr:...","[(-122.034442, 37.3912438)]"
1807,node,10844739568,linkedin building j,office,yes,False,,"addr:city:sunnyvale,addr:housenumber:580,addr:...","[(-122.0363024, 37.3920298)]"
1808,node,10844739569,linkedin,office,yes,False,,"addr:city:sunnyvale,addr:housenumber:880,addr:...","[(-122.0372172, 37.3913742)]"
1941,way,25796511,linkedin,building:use,office,True,commercial,"addr:city:sunnyvale,addr:housenumber:1000,addr...","[(-122.0422986, 37.3927221), (-122.042388, 37...."
1942,way,25796520,linkedin,building:use,office,True,commercial,"addr:city:sunnyvale,addr:housenumber:950,addr:...","[(-122.0412881, 37.3924897), (-122.0412188, 37..."
3343,way,102376955,linkedin,building,office,True,office,"addr:city:sunnyvale,addr:housenumber:845,addr:...","[(-122.0350768, 37.3921368), (-122.0347451, 37..."
3351,way,102376971,linkedin,,,True,commercial,"addr:housenumber:755,addr:street:north mathild...","[(-122.0304461, 37.3944523), (-122.0299343, 37..."


In [36]:
df_osm[df_osm['id']==512681243]

,type,id,name,tag_key,tag_val,is_building,building_tag,tags,points


<h1>Lets do some dedupping if we think its necessary, if a WAY has a node that is already included in the data set, remove the WAY</h1>
Note: this may under estimate the num of offices

In [ ]:
dup_way_and_nodes = []
for _, way in df_osm[df_osm['type'] == 'way'].iterrows():
#     if way.id != 512681243:
#         continue
    if len(way.points) < 3:
        continue
    for _, node in df_osm[df_osm['type'] == 'node'].iterrows():
#         if node.id != 10653099469:
#             continue
        polygon = Polygon(way.points)
        point = Point(node.points[0][0], node.points[0][1])
        if polygon.contains(point):
            dup_way_and_nodes.append([way.id, node.id])
df_way_and_nodes = pd.DataFrame(dup_way_and_nodes, columns=['wid', 'nid'], dtype='int')

In [ ]:
df_way_and_nodes[df_way_and_nodes['wid']==264624898]

In [ ]:
dup_ways = set(df_way_and_nodes['wid'].unique())

In [ ]:
df_offices = df_osm[
    df_osm[['type','id']].apply(
        lambda x:True if x['type']=='node' else True if x['id'] not in dup_ways else False, 
        axis=1
    )
]
df_offices

In [ ]:
df_offices[df_offices.apply(lambda x:'linkedin' in x['name'] or 'linkedin' in x['tags'], axis=1)]